# NLOS

## HHL Implementation

In [18]:
import math
import warnings
import numpy as np
import qiskit
from scipy.linalg import expm
from qiskit.circuit.library import StatePreparation, ExactReciprocalGate, phase_estimation
from qiskit.quantum_info import Statevector, Operator
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.result import Result
from qiskit.circuit import IfElseOp, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import Initialize , UnitaryGate , QFTGate, ExactReciprocalGate

from qiskit.quantum_info.operators.predicates import is_unitary_matrix

In [19]:
def progress_callback(**kwargs):
    pass_name = kwargs['pass_'].name()
    count = kwargs['count']
    # The DAG circuit is an internal representation that can be inspected
    dag = kwargs['dag'] 
    # print(f"Pass {count}: {pass_name}, Operations: {dag.count_ops()}")


def hhl_fidelity(x_classic, probs_hhl):
    # Classical solution
    x = x_classic / np.linalg.norm(x_classic)

    # Rebuild complex vector from probabilities (up to global phase)
    # HHL gives only magnitudes; pick sign so inner product is maximized
    # Just treat as positive real for fidelity purposes

    probs_classical = np.abs(x) ** 2
    probs_classical = probs_classical / probs_classical.sum()

    # Normalize HHL probs just in case
    probs_hhl = probs_hhl / probs_hhl.sum()

    # Probability "fidelity" (Bhattacharyya coefficient)
    fidelity = float(np.sum(np.sqrt(probs_classical * probs_hhl)))
    # Numerical safety for arccos
    fidelity_clamped = max(min(fidelity, 1.0), -1.0)
    angle = float(np.rad2deg(np.arccos(fidelity_clamped)))

    return fidelity, angle

In [ ]:
class HHL:
    """class that implements HHL algorithm"""

    def __init__(self, A: np.typing.NDArray, b: np.typing.NDArray, state_dim: int = -1, tol=1e-05):
        """A: operator matrix, b: matrix, n: number of qubits"""
        assert (len(A.shape) == 2 and len(b.shape) == 1), "Inconsistent dimension of A or b"
        assert (A.shape[0] == b.shape[0]), "shape of A does not match b"
        
        self.A = A.copy()
        self.b = b.copy()

        if (self.A.shape[0] != self.A.shape[1] or not np.allclose(self.A, self.A.conj().T, atol=tol)):
            if (self.A.shape[0] != self.A.shape[1]):
                print("A is not a square matrix")
            else:
                print("A is not Hermitian")
            self.A = np.block([[np.zeros((self.A.shape[0], self.A.shape[0])), self.A],
                               [self.A.conj().T, np.zeros((self.A.shape[1], self.A.shape[1]))]])
            print("A is evaluated as [[0, A], [AT, 0]]")
            print("-----------------------------------")
            self.b = np.block([b, np.zeros(A.shape[1])])
        
        self.n_b = 0
        while (self.A.shape[0] > (1 << self.n_b)):
            self.n_b += 1
        if (self.A.shape[0] != (1 << self.n_b)):
            print("A is not power of 2, pad to make A {0} * {0}".format((1 << self.n_b)))
            print("-----------------------------------")
            pad = (1 << self.n_b) - self.A.shape[0]
            before_pad = self.A.shape[0]

            # todo: avoid computing directly
            eigvals = np.linalg.eigvalsh(self.A)
            # set alpha to avoid eigvals to be 0
            # if eigval is zero, HHL blows up
            alpha = np.sqrt(np.abs(eigvals).min() * np.abs(eigvals).max())

            self.A = np.pad(self.A, ((0, pad), (0, pad)), "constant", constant_values=0)
            self.b = np.pad(self.b, (0, (1 << self.n_b) - self.b.shape[0]), "constant", constant_values=0)

        self.tol = tol
        self.dim = (1 << self.n_b)

        if (not math.isclose(np.linalg.norm(self.b), 1, abs_tol=tol)):
            self.b = self.b.copy() / np.linalg.norm(self.b)
            print(f"b is not normalized, normalized to b = {self.b} during init")
            print("-----------------------------------")
        
        
        # C for making C/λ < 1
        eigvals = np.linalg.eigvalsh(self.A)
        self.C = np.abs(eigvals).min() * 0.9
        # todo: bound eigvals and avoid direct computation

        self.kappa = np.abs(eigvals).max() / np.abs(eigvals).min() # characteristic number of matrix A
        # define t slightly smaller -> then if eigval is positive, < 0.45 and negative, > 0.55
        # if just pi / |lambda_max| -> 0.5 eigval errors
        self.t = np.pi / np.abs(eigvals).max() / 1.5 # calculate t

        if (state_dim > 0):
            self.state_dim = state_dim
        else:
            self.state_dim = int(np.log2(self.kappa)) + 6

        self.U = expm(1j * self.t * self.A) # U = e^(iAt), then convert to unitarygate
    
    def _create_register(self):
        """call this to initially create registers, only once"""
        b_reg = QuantumRegister(self.n_b, 'sys')
        c_reg = QuantumRegister(self.state_dim, 'phase')
        aux_reg = QuantumRegister(1, 'aux')
        classic_aux_reg = ClassicalRegister(1, 'classic_aux')
        result_reg = ClassicalRegister(self.n_b, 'classic_sys')
        return b_reg, c_reg, aux_reg, classic_aux_reg, result_reg


    def _prepare_state(self, circ: QuantumCircuit, b_reg: QuantumRegister):
        """add gate to mount b on registers"""
        # assert math.isclose(np.linalg.norm(self.b), 1, abs_tol=self.tol) # check if b is normalized
        init_gate = StatePreparation(Statevector(self.b))
        circ.append(init_gate, b_reg) # add gate for initalizing b
        circ.barrier() # separation from rest of the process

    def _assign_neg_sign(self, circ, c_reg, sign_reg):
        phase_le = list(reversed(c_reg)) # since qpe gives us big-endian, should reverse/little-endian expected in ExactReciprocalGate
        # if the most-significant bit(msb) is 1 -> phase > 1
        # therefore sign must be 1
        # if msb = 1 -> set sign to 1(controlled-not on sign_reg)
        circ.cx(phase_le[-1], sign_reg[0])
        

    def _controlled_rotation(self, circ: QuantumCircuit, c_reg: QuantumRegister, aux_reg: QuantumRegister, sign_reg: QuantumRegister):
        """add controlled rotation block to circuit"""
        # neg_vals = True: eigvals may be negative
        # if negative, ExactReciprocalGate expects sign qubit to be 1
        # and the phase to be (1 - phase)
        # switch to negative if sign = 1 in _assign_neg_sign
        rot_gate = ExactReciprocalGate(
            num_state_qubits=self.state_dim + 1, 
            scaling=self.C * self.t / (2 * np.pi), 
            neg_vals=True
        )
        circ.append(rot_gate, [*sign_reg, *reversed(c_reg), *aux_reg])
    

    def _qpe(self, circ: QuantumCircuit, b_reg, c_reg, inverse=False):
        """add QPE/IQPE gate to the circuit"""
        qpe_circ = phase_estimation(self.state_dim, Operator(self.U).to_instruction())
        if (inverse):
            qpe_circ = qpe_circ.inverse()
        circ.append(qpe_circ.to_gate(), [*c_reg, *b_reg]) # order may change


    def _measure(self, circ: QuantumCircuit, aux_reg: QuantumRegister, b_reg: QuantumRegister, classic_aux_reg: ClassicalRegister, result_reg: ClassicalRegister):
        """measure the quantum qubits and store in classic bits"""
        circ.measure(aux_reg, classic_aux_reg)
        circ.measure(b_reg, result_reg)

    def transpile(self, simulator=AerSimulator):
        """function to transpile circuit. required to be executed before run"""
        b_reg, c_reg, aux_reg, classic_aux_reg, result_reg = self._create_register()


        circ = QuantumCircuit(aux_reg, c_reg, b_reg, result_reg, classic_aux_reg)
        self._prepare_state(circ, b_reg)
        self._qpe(circ, b_reg, c_reg, inverse=False)
        circ.barrier()
        # check if phase is negative
        self._assign_neg_sign(circ, c_reg, sign_reg)
        circ.barrier()
        self._controlled_rotation(circ, c_reg, aux_reg, sign_reg)
        circ.barrier()
        self._assign_neg_sign(circ, c_reg, sign_reg)
        circ.barrier()
        self._qpe(circ, b_reg, c_reg, inverse=True)
        circ.barrier()
        self._measure(circ, aux_reg,  b_reg, classic_aux_reg, result_reg)

        self.backend = simulator()
        return transpile(circ, backend=self.backend, callback=progress_callback), circ
    
    def run(self, circ, shots=1):
        try:
            return self.backend.run(circ, shots=shots).result()
        except AttributeError:
            raise AttributeError("Circuit is not Transpiled before running")
    
    def transpile_and_run(self, shots=1):
        transpiled_circ = self.transpile()
        return self.run(transpiled_circ, shots=shots).result()
    
    def analyze(self, result: Result, show_counts=False) -> tuple[float, np.ndarray]:
        count = result.get_counts()

        if (isinstance(count, dict)):
            if show_counts:
                print(count)
            success = {res: cnt for res, cnt in count.items() if res.split()[0] == '1'}
            # failure = {res: cnt for res, cnt in count.items() if res.split()[0] == '0'}
            tot = sum(success.values())
            success_rate = tot / sum(count.values())
            
            x = np.zeros((1 << self.n_b), dtype=float)
            for res, cnt in success.items():
                ind = int(res.split()[1], 2)
                x[ind] = cnt / tot
            return success_rate, x
        else:
            print(count)
            raise TypeError("result.get_counts() is not type dict[str, int]")

In [30]:
    # [1, 2, 0, 0],
    # [2, 1, 0, 0],
    # [0, 0, 1, 2],
    # [0, 0, 2, 1],
A = np.array([
    [1],
    [-2],
], dtype=float)
b = np.array([2, -4], dtype=float)


alg = HHL(A, b)
print(np.linalg.eigvalsh(alg.A))

A is not a square matrix
A is evaluated as [[0, A], [AT, 0]]
-----------------------------------
A is not power of 2, pad to make A 4 * 4
-----------------------------------
b is not normalized, normalized to b = [ 0.4472136  -0.89442719  0.          0.        ] during init
-----------------------------------
[-2.23606798e+00  2.24387153e-17  7.08339556e-09  2.23606798e+00]


In [26]:
circ, c = alg.transpile()

ValueError: Input matrix is not unitary.

In [24]:
# classic
x_classic = np.linalg.solve(alg.A, alg.b)

for shots in [10**x for x in range(3, 7)]:
    print(f"shots: {shots}")
    result = alg.run(circ, shots=shots)

    p, x = alg.analyze(result, show_counts=False)
    print(f"success rate: {p}\n{x}")
    fidelity, deg = hhl_fidelity(x_classic, x)
    print(f"Fidelity: {fidelity}, angle: {deg}")



print(x_classic ** 2 / np.linalg.norm(x_classic) ** 2)

shots: 1000
success rate: 0.035
[0.34285714 0.         0.65714286 0.        ]
Fidelity: 0.9747716742743392, angle: 12.897310074940927
shots: 10000
success rate: 0.0431
[0.22969838 0.05568445 0.71461717 0.        ]
Fidelity: 0.998890019636735, angle: 2.69982402572793
shots: 100000
success rate: 0.04538
[0.22300573 0.04297047 0.7340238  0.        ]
Fidelity: 0.9998511686713291, angle: 0.9885307792554274
shots: 1000000
success rate: 0.045248
[0.23479491 0.04210131 0.72310378 0.        ]
Fidelity: 0.999834234433296, angle: 1.0432557595668348
[0.22727273 0.03636364 0.73636364 0.        ]


In [ ]:
import numpy as np

# -------------------------
# Block encoding (Halmos dilation)
# -------------------------

def qsp_U00_batch(xs, phases):
    W_list = [W_signal(x) for x in xs]
    res = []
    for W in W_list:
        U = Rz(phases[0])
        for phi in phases[1:]:
            U = W @ U
            U = Rz(phi) @ U
        res.append(U[0, 0])
    return np.array(res)

def psd_sqrt(M):
    M = np.array(M, dtype=np.complex128)
    M = (M + M.conj().T) / 2
    w, V = np.linalg.eigh(M)
    w = np.clip(w, 0.0, None)
    return V @ np.diag(np.sqrt(w)) @ V.conj().T

def block_encode_halmos(A, alpha=None):
    """
    Returns (alpha, U) where U is 2n x 2n unitary and top-left block is A/alpha.
    A need not be Hermitian/unitary.
    """
    A = np.array(A, dtype=np.complex128)
    n, m = A.shape
    if n != m:
        raise ValueError("A must be square for this simple Halmos block-encoding.")
    if alpha is None:
        alpha = float(np.linalg.norm(A, 2))
        if alpha == 0:
            alpha = 1.0
    Atil = A / alpha
    I = np.eye(n, dtype=np.complex128)

    B = psd_sqrt(I - Atil @ Atil.conj().T)
    C = psd_sqrt(I - Atil.conj().T @ Atil)

    U = np.block([[Atil, B],
                  [C,    -Atil.conj().T]])
    return alpha, U


# -------------------------
# QSP core (2x2 signal)
# -------------------------

def W_signal(x):
    """2x2 signal unitary for QSP; x in [-1,1]."""
    x = float(x)
    if abs(x) > 1 + 1e-12:
        raise ValueError("x must be in [-1,1]")
    s = np.sqrt(max(0.0, 1.0 - x*x))
    return np.array([[x, 1j*s],
                     [1j*s, x]], dtype=np.complex128)

def Rz(phi):
    return np.diag([np.exp(1j*phi), np.exp(-1j*phi)]).astype(np.complex128)

def qsp_U00(x, phases):
    """
    U(x) = Rz(phi0) W(x) Rz(phi1) W(x) ... W(x) Rz(phid)
    return (0,0) entry, which is the QSP polynomial value (complex in general).
    """
    U = Rz(phases[0])
    W = W_signal(x)
    for phi in phases[1:]:
        U = W @ U
        U = Rz(phi) @ U
    return U[0, 0]


# -------------------------
# Target function: truncated inverse on [-1,1]
# -------------------------

def truncated_inverse_target(x, kappa, x0=None, x1=None):
    """
    Odd extension target for QSP:
      - for |x| >= x1: g(x) = 1/(kappa*x)
      - for |x| <= x0: g(x) = 0
      - smooth cubic transition in between (C1-ish, bounded)

    Default: x1 = 1/kappa, x0 = 0.5/kappa
    (so we don't force behavior extremely near 0 where inverse is nasty)
    """
    x = float(x)
    sgn = 1.0 if x >= 0 else -1.0
    ax = abs(x)
    if x1 is None: x1 = 1.0 / kappa
    if x0 is None: x0 = 0.5 / kappa
    if x0 <= 0 or x1 <= x0:
        raise ValueError("Need 0 < x0 < x1.")
    if ax <= x0:
        return 0.0
    if ax >= x1:
        return sgn * (1.0 / (kappa * ax))

    # Smoothstep t in [0,1]
    t = (ax - x0) / (x1 - x0)
    smooth = t*t*(3 - 2*t)  # cubic smoothstep

    # Blend from 0 to 1/(kappa*ax); keep bounded by <= 1
    return sgn * smooth * (1.0 / (kappa * ax))


# -------------------------
# Phase fitter (robust-ish)
# -------------------------

class QSPPhaseFitter:
    """
    Fits phases to approximate a target function g(x) on [-1,1] (or subset),
    with penalties to enforce |p(x)| <= 1 (required for QSP feasibility).
    """

    def __init__(self, degree, kappa, seed=0):
        self.degree = int(degree)
        self.kappa = float(kappa)
        self.rng = np.random.default_rng(seed)

    def _make_grids(self, n_cheb=128, n_dense=1024, focus_min=1.0/self.kappa):
        # Chebyshev nodes on [-1,1]
        j = np.arange(n_cheb)
        x_cheb = np.cos((2*j + 1) * np.pi / (2*n_cheb))

        # Dense uniform grid for safety checks
        x_dense = np.linspace(-1.0, 1.0, n_dense)

        if focus_min is not None:
            # Optionally focus loss on |x| >= focus_min (e.g. 1/kappa zone)
            x_cheb = x_cheb[np.abs(x_cheb) >= focus_min]
            x_dense = x_dense[np.abs(x_dense) >= focus_min]

        return x_cheb, x_dense

    def fit(self,
            target_fn,
            n_cheb=16,
            n_dense=16,
            focus_min=None,
            iters=10,
            lr=0.05,
            penalty_w=50.0,
            bound_eps=1e-6,
            multi_start=6):
        """
        Returns best_phases, info dict.
        - penalty enforces |p(x)|<=1 on dense grid (soft).
        - multi_start tries multiple random initializations and picks the best verified.
        """
        d = self.degree
        best = None
        best_info = None

        x_cheb, x_dense = self._make_grids(n_cheb=n_cheb, n_dense=n_dense, focus_min=focus_min)
        y_cheb = np.array([target_fn(x) for x in x_cheb], dtype=np.float64)
        y_dense = np.array([target_fn(x) for x in x_dense], dtype=np.float64)

        def loss(phases):
            # Primary fit on Chebyshev nodes
            vals = qsp_U00_batch(x_cheb, phases)
            # We want real target; take real part but also penalize imag leakage
            diff_re = (vals.real - y_cheb)
            diff_im = vals.imag
            mse = np.mean(diff_re*diff_re) + 0.2*np.mean(diff_im*diff_im)

            # Bound penalty on dense grid
            vd = qsp_U00_batch(x_dense, phases)
            abs_v = np.abs(vd)
            viol = np.clip(abs_v - (1.0 + bound_eps), 0.0, None)
            pen = penalty_w * np.mean(viol*viol)

            return mse + pen

        def finite_diff_grad(phases, eps=1e-4):
            g = np.zeros_like(phases)
            base = loss(phases)
            for i in range(len(phases)):
                p1 = phases.copy(); p1[i] += eps
                p2 = phases.copy(); p2[i] -= eps
                g[i] = (loss(p1) - loss(p2)) / (2*eps)
            return base, g

        def optimize_from(init):
            phases = init.copy()
            # simple Adam + backtracking-ish
            m = np.zeros_like(phases)
            v = np.zeros_like(phases)
            b1, b2 = 0.9, 0.999
            eps_adam = 1e-8

            cur = loss(phases)
            for t in range(1, iters+1):
                cur, g = finite_diff_grad(phases)
                m = b1*m + (1-b1)*g
                v = b2*v + (1-b2)*(g*g)
                mhat = m / (1 - b1**t)
                vhat = v / (1 - b2**t)
                step = lr * mhat / (np.sqrt(vhat) + eps_adam)

                # backtracking to avoid explosions
                new_phases = phases - step
                new_loss = loss(new_phases)
                if new_loss <= cur:
                    phases = new_phases
                    cur = new_loss
                else:
                    # shrink step
                    phases = phases - 0.3*step
                    cur = loss(phases)

                # keep phases in a reasonable range (wrap)
                phases = (phases + np.pi) % (2*np.pi) - np.pi

            return phases, cur

        def verify(phases):
            # Check max |p(x)| on full dense [-1,1]
            xs = np.linspace(-1.0, 1.0, 2048)
            vals = np.array([qsp_U00(x, phases) for x in xs])
            max_abs = float(np.max(np.abs(vals)))
            # Error on inverse region |x| >= 1/kappa
            mask = np.abs(xs) >= (1.0 / self.kappa)
            err = float(np.max(np.abs(vals.real[mask] - np.array([target_fn(x) for x in xs[mask]]))))
            imag_max = float(np.max(np.abs(vals.imag)))
            return max_abs, err, imag_max

        for s in range(multi_start):
            init = self.rng.normal(scale=0.2, size=d+1)
            phases, L = optimize_from(init)
            max_abs, err, imag_max = verify(phases)

            # Prefer feasible-ish and low error
            score = L + 10.0*max(0.0, max_abs-1.0) + err + 0.1*imag_max
            info = {"loss": L, "max_abs": max_abs, "max_err_on_[1/k,1]": err, "max_imag": imag_max}

            if best is None or score < best_info["score"]:
                best = phases
                best_info = {"score": score, **info}

        return best, best_info


# -------------------------
# Reliable-ish solver: build block encoding + fit QSP phases + apply via SVD (trustworthy SVT action)
# -------------------------

class ReliableQSVTInverseApprox:
    """
    - Builds block encoding (for completeness / later circuit work).
    - Fits QSP phases to a truncated inverse target.
    - Applies the resulting polynomial as a singular value transform using SVD (reliable numerically).
    """

    def __init__(self, A, kappa, degree=9, seed=0):
        self.A = np.array(A, dtype=np.complex128)
        n, m = self.A.shape
        if n != m:
            raise ValueError("This minimal solver assumes square A.")
        self.n = n
        self.kappa = float(kappa)
        if self.kappa < 1:
            raise ValueError("kappa must be >= 1")

        # Block encoding
        self.alpha, self.U_block = block_encode_halmos(self.A)

        # Fit phases
        self.fitter = QSPPhaseFitter(degree=degree, kappa=self.kappa, seed=seed)

        # Target function
        self.target_fn = lambda x: truncated_inverse_target(x, self.kappa)

        self.phases = None
        self.fit_info = None

    def fit_phases(self,
                   n_cheb=64,
                   n_dense=128,
                   iters=600,
                   lr=0.05,
                   penalty_w=80.0,
                   multi_start=8):
        # focus_min: don't overfit near zero; QLSA cares about |x|>=1/kappa region
        phases, info = self.fitter.fit(
            target_fn=self.target_fn,
            n_cheb=n_cheb,
            n_dense=n_dense,
            focus_min=None,
            iters=iters,
            lr=lr,
            penalty_w=penalty_w,
            multi_start=multi_start
        )
        self.phases = phases
        self.fit_info = info
        return phases, info

    def p(self, x):
        """QSP polynomial value (real part is what we target)."""
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")
        return qsp_U00(x, self.phases)

    def apply_inverse_approx(self, b):
        """
        Applies approximate inverse using singular value transformation:
          A^{-1} b ≈ (kappa/alpha) * U diag(p(sigma/alpha)) V^† b
        where p approximates g(x)=1/(kappa x) on [1/kappa,1] with truncation near 0.
        """
        if self.phases is None:
            raise RuntimeError("Call fit_phases() first.")

        b = np.array(b, dtype=np.complex128).reshape(-1)
        if b.shape[0] != self.n:
            raise ValueError("b has wrong dimension")

        U, s, Vh = np.linalg.svd(self.A, full_matrices=False)
        x = s / self.alpha  # normalized singular values in [0,1]

        # Evaluate p(x) on singular values (use real part)
        px = np.array([self.p(xi).real for xi in x], dtype=np.float64)

        # Scale back to approximate 1/s
        # p(x) ≈ 1/(kappa x) => (kappa/alpha)*p(s/alpha) ≈ 1/s
        scale = (self.kappa / self.alpha)
        inv_diag = scale * px

        return U @ (inv_diag * (Vh @ b))

    def diagnostics(self):
        if self.fit_info is None:
            return None
        return {
            "alpha": self.alpha,
            "fit_info": self.fit_info,
        }


In [55]:
import numpy as np

# 안정적인 A 만들기 (SPD)
# M = np.random.randn(6, 6)
# A = M.T @ M + 0.5*np.eye(6)

A = np.array([[1,0,0],
              [0,1,0],
              [0,0,1]], dtype=float)

condA = np.linalg.cond(A, 2)
print("cond(A) =", condA)

b = np.random.randn(3)

solver = ReliableQSVTInverseApprox(A, kappa=condA*1.1, degree=9, seed=0)
phases, info = solver.fit_phases(n_cheb=64,n_dense=64,iters=200,lr=0.05,penalty_w=60.0,multi_start=3)
np.save("phases_kappa5_deg9.npy", phases)
print("fit:", info)

x_hat = solver.apply_inverse_approx(b)
x_true = np.linalg.solve(A, b)

print(x_true)
print(x_hat)
print("err =", np.linalg.norm(x_hat - x_true))
print("max|Im| =", np.max(np.abs(x_hat.imag)))

# phases = np.load("phases_kappa5_deg9.npy")

# solver = ReliableQSVTInverseApprox(A, kappa=kappa, degree=9, seed=0)
# solver.phases = phases   # fit_phases() 안 돌리고 바로 주입
# x_hat = solver.apply_inverse_approx(b)




cond(A) = 1.0
fit: {'score': np.float64(0.09748876523693553), 'loss': np.float64(0.0020467348348851295), 'max_abs': 1.0, 'max_err_on_[1/k,1]': 0.0825035480334676, 'max_imag': 0.129384823685828}
[ 1.13747378  0.59109272 -1.03896283]
[ 1.24070396+0.j  0.64473669+0.j -1.13325277+0.j]
err = 0.14974891642541505
max|Im| = 0.0
